<a href="https://colab.research.google.com/github/geovalexis/TFG/blob/main/2_Matrix_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Matrix preparation

In [ ]:
# Basic imports
import numpy as np
import pandas as pd

## Homo Sapiens orthologs filtering

Retrieve only those orthologs for Homo Sapiens in the Orthologs dataset chosen


In [ ]:
mtp_orthologs = pd.read_csv("drive/MyDrive/TFG/MtP_201912.tab.tsv", sep="\t", names=["protein1", "protein2"])
qfo_ref_human_proteome = pd.read_csv("drive/MyDrive/TFG/QFO_2018/human_reference_proteome.tsv", sep="\t", names=["protein1"])
print("Total size of the orthologs file from MetaPhOrs:", mtp_orthologs.size)
print("Number of reference proteins (from QfO):", qfo_ref_human_proteome.size)
human_orthologs = pd.merge(mtp_orthologs, qfo_ref_human_proteome, how="inner", on="protein1")
print("Size of the resulting inner join between the two datasets:", human_orthologs.size)
print("Current number of Homo Sapiens reference proteins within MetaPhOrs:", human_orthologs["protein1"].unique().size)
#human_orthologs.drop_duplicates() # There just around 200 repeated rows but I think that they correspond to the orthologs within the same specie (Homo sapiens in this case) -> diagonal
human_orthologs

Total size of the orthologs file from MetaPhOrs: 7898268
Number of reference proteins (from QfO): 20996
Size of the resulting inner join between the two datasets: 139718
Current number of Homo Sapiens reference proteins within MetaPhOrs: 5254


,protein1,protein2
0,Q8ND71,Q7ZAM9
1,Q8ND71,Q6CDT3
2,Q8ND71,A2ESR8
3,Q8ND71,A2E1H0
4,Q8ND71,A2FTJ3
...,...,...
69854,P20930,A7T7C6
69855,P20930,A7T7U2
69856,A0A1B0GTG1,A9UR99
69857,Q12923,A9VDP5


## TaxIDs mapping

### Locally 

Several options here:
* From Uniprot database (**the one actually used**)
* From NCBI database 
* From GOA database 

#### Uniprot 
* From the file `idmapping_selected_subset.tab.gz`.
* Note that `selected` refers to a selection of databases' Ids, not species.

In [ ]:
# IMPORTANT:  This code will not work with the original `idmapping_selected_subset.tab.gz` file.
#             See '1_Data_Retrieval' notebook, section 'Id mapping file', to learn how this idmapping file was downloaded and filter in order to save memory and disk storage.

def mapUniprot2Taxid_Uniprot(uniprotIDs: set, idmapping) -> Dict: 
  """ Translate a set of genes to its corresponding taxIDs given their uniprotKB accession numbers.
  Args:
      uniprotIDs (set): set of genes to translate
      idmapping (os.path): path to the idmapping_selected.tab gzip compressed file 
                          from the official Uniprot ftp server.
                          See ftp://ftp.ebi.ac.uk/pub/databases/uniprot/current_release/knowledgebase/idmapping/ 
  """    
  uniprot2taxid = pd.read_table(idmapping,
                          compression="gzip", 
                          names=["UniprotKB-AC", "GO", "NCBI-taxon"], 
                          usecols=["UniprotKB-AC", "NCBI-taxon"],
                          dtype={"UniprotKB-AC":"string", "NCBI-taxon": "int32"})
  uniprot2taxid = uniprot2taxid[uniprot2taxid["UniprotKB-AC"].isin(uniprotIDs)] 
  # Check if all the uniprotIDs have a corresponding taxID
  if (uniprot2taxid["UniprotKB-AC"].nunique() != len(uniprotIDs)):
      print(f"The tax IDs for {len(uniprotIDs)-uniprot2taxid['UniprotKB-AC'].nunique()} uniprotKB accession numbers couldn't be found.")
  return uniprot2taxid.set_index("UniprotKB-AC")["NCBI-taxon"].to_dict()

#### NCBI database

NCBI also provides a file for differents databases IDs mapping: https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/accession2taxid/prot.accession2taxid.gz 

**Not used** because the file only contained the ID mapping for live protein sequence records and there were a lot that were lacking (they are probable in the other file called `dead_prot.accession2taxid.gz`

In [ ]:
def mapUniprot2Taxid_NCBI(uniprotIDs: set, prot_accession2taxid) -> Dict:
    """ Translate a set of genes to its corresponding taxIDs given their uniprotKB accession numbers.
    Args:
        uniprotIDs (set): set of genes to translate
        prot_accession2taxid (os.path): path to the prot.accession2taxid gzip compressed file 
                                        from the official NCBI ftp server.
                                        See ftp.ncbi.nlm.nih.gov/pub/taxonomy/accession2taxid/ 
    """    
    uniprot2taxid = pd.read_table(prot_accession2taxid,
                            compression="gzip", 
                            names=["accession", "accession.version", "taxid", "gi"],
                            usecols=["accession", "taxid"],
                            dtype={"accession":"string", "taxid": "int32"})
    uniprot2taxid = uniprot2taxid[uniprot2taxid["accession"].isin(uniprotIDs)] 
    # Check if all the uniprotIDs have a corresponding taxID
    if (uniprot2taxid["accession"].nunique() != len(uniprotIDs)):
        print(f"The tax ID for {len(uniprotIDs)-uniprot2taxid['accession'].nunique()} uniprotKB accession numbers couldn't be found.")
        print(set.difference(uniprotIDs, set(uniprot2taxid["accession"].unique())))
    return uniprot2taxid.set_index("accession")["taxid"].to_dict()

#### GOA database 
 
 We could also used the file ftp://ftp.ebi.ac.uk/pub/databases/GO/goa/UNIPROT/goa_uniprot_all.gpi.gz 

 **Not used** simply because the Uniprot idmapping file worked pretty well

In [ ]:
#Not implemented

## GO terms assignment

### Retrievement

#### By external APIs

We could extend the "function map_uniprotIDs2taxIDs_EBIRequest" to also retrieve the GO ids -> We couldn't apply the filters from here

In [ ]:
#Not Implemented

#### Locally (by a GAF file)

In [ ]:
# IMPORTANT: see first notebook '1_Data_Retrieval' to see how the original GAF file was filter by QfO taxa to save memory and disk storage. 
#             In this case we used the GAF file for human just for demonstrative purposes.

go_annotations = pd.read_csv("goa_human.gaf.gz", 
                             sep="\t", 
                             header=None, 
                             names=["DB", "DB_Object_ID", "DB_Object_Symbol", "Qualifier", "GO_ID", "DB:Reference", "Evidence Code", "With (or) From", "Aspect", "DB_Object_Name", "DB_Object_Synonym", "DB_Object_Type", "Taxon and Interacting taxon", "Date","Assigned_By", "Annotation_Extension", "Gene_Product_Form_ID"],
                             skiprows=12, 
                             compression="gzip")
go_annotations


,DB,DB_Object_ID,DB_Object_Symbol,Qualifier,GO_ID,DB:Reference,Evidence Code,With (or) From,Aspect,DB_Object_Name,DB_Object_Synonym,DB_Object_Type,Taxon and Interacting taxon,Date,Assigned_By,Annotation_Extension,Gene_Product_Form_ID
0,UniProtKB,A0A024R1R8,hCG_2014768,NaN,GO:0002181,PMID:21873635,IBA,PANTHER:PTN002008372|SGD:S000007246,P,Coiled-coil domain-containing protein 72,hCG_2014768,protein,taxon:9606,20171102,GO_Central,NaN,NaN
1,UniProtKB,A0A024RBG1,NUDT4B,NaN,GO:0000298,PMID:21873635,IBA,PANTHER:PTN000290327|SGD:S000005689,F,Diphosphoinositol polyphosphate phosphohydrola...,NUDT4B,protein,taxon:9606,20170228,GO_Central,NaN,NaN
2,UniProtKB,A0A024RBG1,NUDT4B,NaN,GO:0003723,GO_REF:0000043,IEA,UniProtKB-KW:KW-0694,F,Diphosphoinositol polyphosphate phosphohydrola...,NUDT4B,protein,taxon:9606,20201128,UniProt,NaN,NaN
3,UniProtKB,A0A024RBG1,NUDT4B,NaN,GO:0005634,PMID:21873635,IBA,FB:FBgn0036111|PANTHER:PTN000290326,C,Diphosphoinositol polyphosphate phosphohydrola...,NUDT4B,protein,taxon:9606,20170228,GO_Central,NaN,NaN
4,UniProtKB,A0A024RBG1,NUDT4B,NaN,GO:0005737,PMID:21873635,IBA,FB:FBgn0036111|PANTHER:PTN000290326|TAIR:locus...,C,Diphosphoinositol polyphosphate phosphohydrola...,NUDT4B,protein,taxon:9606,20170228,GO_Central,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607386,UniProtKB,W6CW81,PYDC5,NaN,GO:0031333,PMID:24531343,IDA,NaN,P,Pyrin domain-containing protein 5,PYDC5|POP3,protein,taxon:9606,20180903,UniProt,NaN,NaN
607387,UniProtKB,W6CW81,PYDC5,NaN,GO:0035458,PMID:21873635,IBA,MGI:MGI:101847|MGI:MGI:1347080|MGI:MGI:2138243...,P,Pyrin domain-containing protein 5,PYDC5|POP3,protein,taxon:9606,20200808,GO_Central,NaN,NaN
607388,UniProtKB,W6CW81,PYDC5,NaN,GO:0035458,PMID:24531343,IDA,NaN,P,Pyrin domain-containing protein 5,PYDC5|POP3,protein,taxon:9606,20180903,UniProt,NaN,NaN
607389,UniProtKB,W6CW81,PYDC5,NaN,GO:0097169,PMID:21873635,IBA,MGI:MGI:2686159|PANTHER:PTN001385767|UniProtKB...,C,Pyrin domain-containing protein 5,PYDC5|POP3,protein,taxon:9606,20190213,GO_Central,NaN,NaN


### Filter GAF file

If we perform this filter before the GO annotations assignment the algorithm would take less time

In [ ]:
# APPLY FILTERS

# By ontology
go_annotations = go_annotations[go_annotations["Aspect"]=="P"] # P (biological process), F (molecular function) or C (cellular component)

# By evidence code. Based on https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3536626/ (in Materials and Methods section) and http://geneontology.org/docs/guide-go-evidence-codes/ 
experimental_evid_codes = {"EXP", "IMP", "HMP", "IGI", "HGI", "IPI", "IEP", "HEP", "IDA", "HDA"} # The ones cited in the paper plus the corresponding "high throughput" version of them
computational_evid_codes = {"ISS", "RCA"}
author_statement_evid_codes = {"NAS", "TAS"}
curator_statement_evid_codes = {"IC"}
evidence_codes_filter = experimental_evid_codes | computational_evid_codes | author_statement_evid_codes | curator_statement_evid_codes # the "|" means "Set Union"
go_annotations = go_annotations[go_annotations["Evidence Code"].isin(evidence_codes_filter)]
go_annotations.to_csv("drive/MyDrive/TFG/goa_human_filtered.gaf", sep="\t", index=False, header=True)
go_annotations



,DB,DB_Object_ID,DB_Object_Symbol,Qualifier,GO_ID,DB:Reference,Evidence Code,With (or) From,Aspect,DB_Object_Name,DB_Object_Synonym,DB_Object_Type,Taxon and Interacting taxon,Date,Assigned_By,Annotation_Extension,Gene_Product_Form_ID
182,UniProtKB,A0A075B6P5,IGKV2-28,NaN,GO:0006898,Reactome:R-HSA-2173782,TAS,NaN,P,Immunoglobulin kappa variable 2-28,IGKV2-28,protein,taxon:9606,20181121,Reactome,NaN,NaN
184,UniProtKB,A0A075B6P5,IGKV2-28,NaN,GO:0006956,Reactome:R-HSA-166663,TAS,NaN,P,Immunoglobulin kappa variable 2-28,IGKV2-28,protein,taxon:9606,20181121,Reactome,NaN,NaN
185,UniProtKB,A0A075B6P5,IGKV2-28,NaN,GO:0006958,Reactome:R-HSA-173623,TAS,NaN,P,Immunoglobulin kappa variable 2-28,IGKV2-28,protein,taxon:9606,20181121,Reactome,NaN,NaN
187,UniProtKB,A0A075B6P5,IGKV2-28,NaN,GO:0030449,Reactome:R-HSA-977606,TAS,NaN,P,Immunoglobulin kappa variable 2-28,IGKV2-28,protein,taxon:9606,20181121,Reactome,NaN,NaN
188,UniProtKB,A0A075B6P5,IGKV2-28,NaN,GO:0038095,Reactome:R-HSA-2454202,TAS,NaN,P,Immunoglobulin kappa variable 2-28,IGKV2-28,protein,taxon:9606,20181120,Reactome,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607329,UniProtKB,Q9Y6Z7,COLEC10,NaN,GO:1904888,PMID:28301481,IMP,NaN,P,Collectin-10,COLEC10|CLL1|UNQ366/PRO702,protein,taxon:9606,20170519,UniProt,NaN,NaN
607378,UniProtKB,W5XKT8,SPACA6,NaN,GO:0007342,GO_REF:0000024,ISS,UniProtKB:E9Q8Q8,P,Sperm acrosome membrane-associated protein 6,SPACA6|SPACA6P|UNQ2487/PRO5774,protein,taxon:9606,20200609,UniProt,NaN,NaN
607386,UniProtKB,W6CW81,PYDC5,NaN,GO:0031333,PMID:24531343,IDA,NaN,P,Pyrin domain-containing protein 5,PYDC5|POP3,protein,taxon:9606,20180903,UniProt,NaN,NaN
607388,UniProtKB,W6CW81,PYDC5,NaN,GO:0035458,PMID:24531343,IDA,NaN,P,Pyrin domain-containing protein 5,PYDC5|POP3,protein,taxon:9606,20180903,UniProt,NaN,NaN


In [ ]:

# Link human genes and GO annotations
go_annotations = pd.read_csv("drive/MyDrive/TFG/goa_human_filtered.gaf", sep="\t", header=0)
human_genes = pd.DataFrame({"human_gene": human_orthologs["protein1"].unique()})
human_goa_subset = go_annotations[["DB_Object_ID", "DB_Object_Symbol", "GO_ID", "DB:Reference", "Evidence Code", "DB_Object_Name", "DB_Object_Synonym", "DB_Object_Type",  "Date"]]
human_genes2GOterms_allAttributes = pd.merge(human_genes, human_goa_subset, left_on="human_gene", right_on="DB_Object_ID", how="left")
human_genes2GOterms_allAttributes.drop("DB_Object_ID", axis=1, inplace=True)

# Count NAs and make dictionary
print(f"There are {human_genes2GOterms_allAttributes['GO_ID'].isna().sum()} genes out of  {human_genes2GOterms_allAttributes['human_gene'].unique().size} that does not have any GO term")
    
# Make it more human readable
human_genes2GOterms_allAttributes.set_index(["human_gene", "GO_ID"], inplace=True)
human_genes2GOterms_allAttributes


There are 2569 genes out of  5254 that does not have any GO term


DB_Object_Symbol  ...        Date
human_gene GO_ID                        ...            
Q8ND71     NaN                     NaN  ...         NaN
P15170     GO:0000082            GSPT1  ...  20041217.0
           GO:0000184            GSPT1  ...  20041217.0
           GO:0000184            GSPT1  ...  20181122.0
           GO:0006449            GSPT1  ...  20190627.0
...                                ...  ...         ...
Q12923     GO:0006470           PTPN13  ...  20190910.0
           GO:0006661           PTPN13  ...  20181121.0
           GO:0014066           PTPN13  ...  20141015.0
           GO:0035335           PTPN13  ...  20141015.0
           GO:0071345           PTPN13  ...  20180419.0

[19505 rows x 7 columns]

In [ ]:
# Save annotations in the json file

import json

human_genes2GOterms = human_genes2GOterms_allAttributes.groupby("human_gene", dropna=True)["GO_ID"].apply(lambda x: list(set(x)) if pd.notna(x).all() else []).to_dict() #  DEPRECATED: apply(set).apply(list).to_dict() # First apply "set" to avoid duplicated GO ids and the to "list" so it can be transformed into json

with open("drive/MyDrive/TFG/human_genes2GOtermIDs.json", "w") as output:
    json.dump(human_genes2GOterms, output, indent=4)

# Deprecated

## Other ways of TaxIDs mapping

### By brute force

This means by directly looking for the taxID within the data retrieved from QfO.
Not used because it was highly inneficient.

In [ ]:
def map_uniprotIDs2taxIDs_BruteForce(uniprotIDs):
  qfo_uniprotIDs = pd.read_csv("drive/MyDrive/TFG/QFO_2018/QfO_uniprotKBs.tsv", sep="\t", header=0, dtype="string")
  qfo_uniprotIDs.fillna("-", inplace=True) #To solve "boolean value of NA is ambiguous pandas" error
  taxIDs = []
  for uniprotID in uniprotIDs:
    found = False
    for taxID in qfo_uniprotIDs:
        if uniprotID in qfo_uniprotIDs[taxID].unique():
            taxIDs.append(taxID)
            found = True
            break
    if not found:
        taxIDs.append(np.nan)
  return taxIDs

### By external (EBI-EMBL, Uniprot) APIs

Not used in the pipeline because MareNostrum 4 does not have internet connection in the computation nodes.

In [ ]:
# Install neccesary tools and import libraries
!pip install aiohttp
!pip install asyncio

import requests
import aiohttp
import asyncio

# # TO SOLVE "This event loop is already running in python" error
# !pip install nest_asyncio
# import nest_asyncio
# nest_asyncio.apply()


In [ ]:
# DIFFERENT WAYS OF LOOKING FOR THE TAXID FROM THE UNIPROTID

%%time

def map_uniprotIDs2taxIDs_UniprotRequest(uniprotIDs):
    # Documentation in https://www.uniprot.org/help/api_queries
    taxIDs = []
    endpoint = "https://www.uniprot.org/uniprot/"
    for uniprotID in uniprotIDs:
        params = {
            'query': uniprotID,
            'columns': 'organism-id', #Other information can be added (just add the field in comma-separated). Ex. 'id,organism-id,genes'
            'format': 'tab'
        }
        response = requests.get(endpoint, params=params)
        if response.status_code == 200:
            taxIDs.append(response.text.splitlines()[1])
        else:
            print(f"The tax for {uniprotID} couldn't be found")
            taxIDs.append(np.nan)
    return taxIDs

def map_uniprotIDs2taxIDs_EBIRequest(uniprotIDs):
    # Documentation in https://www.ebi.ac.uk/proteins/api/doc/
    taxIDs = []
    for uniprotID in uniprotIDs:
        requestURL = f"https://www.ebi.ac.uk/proteins/api/proteins/{uniprotID}"
        response = requests.get(requestURL, headers={ "Accept" : "application/json"})
        if response.ok: # status_code == 200
            taxIDs.append(response.json()["organism"]["taxonomy"])
        else:
            print(f"The tax for {uniprotID} couldn't be found")
            taxIDs.append(np.nan)
    return taxIDs

print(map_uniprotIDs2taxIDs_EBIRequest(["Q7ZAM9", "Q8F6D0", "Q8F6A3", "Q8F692", "Q8F624", "Q7L211", "Q8CXT3"]))


[189518, 189518, 189518, 189518, 189518, 9606, 189518]
CPU times: user 84.4 ms, sys: 4.14 ms, total: 88.5 ms
Wall time: 5.51 s


In [ ]:
# MULTIPROCESSING VERSION OF map_uniprotIDs2taxIDs_EBIRequest -> The sequential implementatio is too slow

async def map_uniprotIDs2taxIDs_EBIRequest(uniprotID, session):
    # Documentation in https://www.ebi.ac.uk/proteins/api/doc/
    while True:
        try:
            requestURL = f"https://www.ebi.ac.uk/proteins/api/proteins/{uniprotID}"
            response = await session.get(requestURL, headers={ "Accept" : "application/json"})
            if response.ok: # status_code == 200
                return {uniprotID:(await response.json())["organism"]["taxonomy"]}
            else:
                #print(f"Protein {uniprotID} raised a {response.status} status code.") # It means that the protein couldn't be found or has been deleted.
                return {}
        except aiohttp.ClientConnectionError as e:
            print(f"Raised a ClientConnectionError: {e.message}")
            asyncio.sleep(0.1)

async def map_uniprotIDs2taxIDs_EBIRequest_multiprocessing(uniprotIDs, chunk=200): # EBI limits requests to 200 requests/second/user
  result = {}
  async with aiohttp.ClientSession() as session:
    if len(uniprotIDs)>chunk:
        for i in range(0, len(uniprotIDs), chunk):
            step = i+chunk if (i+chunk) < len(uniprotIDs) else len(uniprotIDs)
            #print(f"{i} proteins already processed. Processing next batch...")
            res_batch = await asyncio.gather(*[map_uniprotIDs2taxIDs_EBIRequest(uniprotID, session) for uniprotID in uniprotIDs[i:step]])
            for j in res_batch:
                result.update(j)
            #sleep(1)
    else:
        res_batch = await asyncio.gather(*[map_uniprotIDs2taxIDs_EBIRequest(uniprotID, session) for uniprotID in uniprotIDs])
        for j in res_batch:
            result.update(j)
  if (len(result) != len(uniprotIDs)):
    print(f"The tax ID for {len(uniprotIDs)-len(result.keys())} uniprotKB accession numbers couldn't be found.")
    #print(set.intersection(uniprotIDs, result.keys()))
  return result

In [ ]:
# For testing
%%time

uniprotIDs = pd.Series(["Q7ZAM9", "Q8F6D0", "NO_EXIST", "Q8F692", "Q8F624", "Q7L211", "Q8CXT3"])
loop = asyncio.get_event_loop()
uniprotID2taxIDs = loop.run_until_complete(map_uniprotIDs2taxIDs_EBIRequest_multiprocessing(uniprotIDs.unique().tolist(), 3))
uniprotIDs = uniprotIDs.apply(lambda x:  uniprotID2taxIDs.get(x))
#uniprotIDs = uniprotIDs.astype(pd.Int64Dtype())
print(uniprotIDs)

In [ ]:
%%time
# With real dataset
loop = asyncio.get_event_loop()
uniprotID2taxIDs = loop.run_until_complete(map_uniprotIDs2taxIDs_EBIRequest_multiprocessing(human_orthologs["protein2"].unique().tolist()))
human_orthologs_withTaxIDs = human_orthologs.copy()
human_orthologs_withTaxIDs["ortholog_taxID"] = human_orthologs["protein2"].apply(lambda x:  uniprotID2taxIDs.get(x)).astype(pd.Int64Dtype()) 
human_orthologs_withTaxIDs.rename({"protein1": "human_gene", "protein2": "ortholog"}, axis=1, inplace=True)
human_orthologs_withTaxIDs = human_orthologs_withTaxIDs[["human_gene", "ortholog", "ortholog_taxID"]]
human_orthologs_withTaxIDs.to_csv("drive/MyDrive/TFG/human_orthologs2taxIDs.tsv", sep="\t", index=False, header=True)

CPU times: user 1min 4s, sys: 5.48 s, total: 1min 9s
Wall time: 14min 17s


In [ ]:
# Checking
print(f"There are {human_orthologs_withTaxIDs['human_gene'].unique().size} human proteins and {human_orthologs_withTaxIDs['ortholog'].size} orthologs.")
print(f"There are {human_orthologs_withTaxIDs['ortholog_taxID'].isna().sum()} proteins that have been deleted or deprecated from the Uniprot database")
# Se podría buscar los taxIDs de estas proteinas en el fichero de QfO_uniprotKBs.tsv (aquí seguro que están) mediante la funcion map_uniprotIDs2taxIDs_BruteForce()
# Aunque si no es una cifra significativa tal vez no sea necesario o bien incluso sería mejor ignorarlas dado que por algo han sido eliminadas
human_orthologs_withTaxIDs

There are 5254 human proteins and 69859 orthologs.
There are 1618 proteins that have been deleted or deprecated from the Uniprot database


,human_gene,ortholog,ortholog_taxID
0,Q8ND71,Q7ZAM9,189518
1,Q8ND71,Q6CDT3,284591
2,Q8ND71,A2ESR8,5722
3,Q8ND71,A2E1H0,5722
4,Q8ND71,A2FTJ3,5722
...,...,...,...
69854,P20930,A7T7C6,45351
69855,P20930,A7T7U2,45351
69856,A0A1B0GTG1,A9UR99,81824
69857,Q12923,A9VDP5,81824


## Create an orthologs matrix with taxIDs and GO terms

In [ ]:
# Assign GO terms IDs to the human reference proteome
human_orthologs_withGOids = pd.read_csv("drive/MyDrive/TFG/human_orthologs2taxIDs.tsv", sep="\t", header=0, dtype={"human_gene": "string", "ortholog":"string", "ortholog_taxID":pd.Int64Dtype()}) 
human_orthologs_withGOids["human_gene_GO_IDs"] = human_orthologs_withGOids["human_gene"].apply(lambda x: human_genes2GOterms.get(x))
human_orthologs_withGOids = human_orthologs_withGOids[["human_gene", "human_gene_GO_IDs", "ortholog", "ortholog_taxID"]]
human_orthologs_withGOids.to_csv("drive/MyDrive/TFG/human_orthologs_MtP-QfO2018.tsv", sep="\t", header=True, index=False)
human_orthologs_withGOids

,human_gene,human_gene_GO_IDs,ortholog,ortholog_taxID
0,Q8ND71,[],Q7ZAM9,189518
1,Q8ND71,[],Q6CDT3,284591
2,Q8ND71,[],A2ESR8,5722
3,Q8ND71,[],A2E1H0,5722
4,Q8ND71,[],A2FTJ3,5722
...,...,...,...,...
69854,P20930,"[GO:0030216, GO:0018149, GO:0061436, GO:007026...",A7T7C6,45351
69855,P20930,"[GO:0030216, GO:0018149, GO:0061436, GO:007026...",A7T7U2,45351
69856,A0A1B0GTG1,[],A9UR99,81824
69857,Q12923,"[GO:0035335, GO:0006470, GO:0001933, GO:001406...",A9VDP5,81824


In [ ]:
human_orthologs_withGOids.iloc[69854]["human_gene_GO_IDs"]

['GO:0030216', 'GO:0018149', 'GO:0061436', 'GO:0070268', 'GO:0007275']